In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar e pré-processar os dados CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalizar os valores de pixel para o intervalo [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Converter rótulos para one-hot encoding (necessário para categorical_crossentropy)
# Se você usar sparse_categorical_crossentropy, não precisa desta linha
# train_labels_one_hot = to_categorical(train_labels, num_classes=10)
# test_labels_one_hot = to_categorical(test_labels, num_classes=10)

print(f"Shape das imagens de treino: {train_images.shape}")
print(f"Shape dos rótulos de treino: {train_labels.shape}") # ou train_labels_one_hot.shape se one-hot

# Construir a CNN do zero
model_cnn_scratch = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # 10 classes para CIFAR-10
])

# Compilar o modelo
model_cnn_scratch.compile(optimizer='adam',
                          loss='sparse_categorical_crossentropy', # Use 'categorical_crossentropy' se usar one-hot encoding
                          metrics=['accuracy'])

# Resumo do modelo
model_cnn_scratch.summary()

# Treinar o modelo
history_cnn_scratch = model_cnn_scratch.fit(train_images, train_labels,
                                            epochs=10, # Você pode ajustar o número de épocas
                                            validation_data=(test_images, test_labels))

# Avaliar o modelo
test_loss_cnn_scratch, test_acc_cnn_scratch = model_cnn_scratch.evaluate(test_images, test_labels, verbose=2)
print(f"\nAcurácia da CNN do zero no teste: {test_acc_cnn_scratch * 100:.2f}%")

# Previsões para a matriz de confusão
y_pred_cnn_scratch = np.argmax(model_cnn_scratch.predict(test_images), axis=1)
conf_matrix_cnn_scratch = confusion_matrix(test_labels, y_pred_cnn_scratch)

print("\nMatriz de Confusão (CNN do Zero):")
print(conf_matrix_cnn_scratch)

# Opcional: Visualizar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_cnn_scratch, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f'Classe {i}' for i in range(10)],
            yticklabels=[f'Classe {i}' for i in range(10)])
plt.title('Matriz de Confusão - CNN do Zero')
plt.ylabel('Rótulo Verdadeiro')
plt.xlabel('Rótulo Previsto')
plt.show()

# Guarde test_acc_cnn_scratch e conf_matrix_cnn_scratch para o relatório

Estratégia 2: Extrator de Características com Rede Pré-Treinada
Aqui, usaremos uma rede pré-treinada (VGG16 é um bom ponto de partida) para extrair características e, em seguida, treinar um classificador simples (como uma Dense ou SVC) sobre essas características.

In [2]:
# --- 0. Instalação e Importações Necessárias ---
# Execute esta célula no seu Google Colab
!pip install transformers datasets accelerate evaluate scikit-learn matplotlib seaborn torch torchvision opencv-python Pillow

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset as HfDataset # Importar Dataset do Hugging Face

# Certifique-se de que o PyTorch está usando a GPU (se disponível)
device = torch.device("cuda" if torch.cuda.enable_gpu() else "cpu") # Alterado de torch.cuda.is_available() para torch.cuda.enable_gpu()
print(f"Usando dispositivo: {device}")

# --- 1. Carregar e Preparar a Base de Dados (Simulação com CIFAR-10) ---
# Como você mencionou "10 classes, 1000 imagens",
# vamos SIMULAR essa situação pegando APENAS 100 imagens por classe do CIFAR-10.
# Em um cenário real, você carregaria suas próprias 1000 imagens.

(train_images_cifar, train_labels_cifar), (test_images_cifar, test_labels_cifar) = datasets.cifar10.load_data()

# Filtrar para ter 100 imagens por classe para simular sua base de 1000 imagens de treino
num_samples_per_class = 100
train_indices = []
for i in range(10): # Para cada uma das 10 classes
    class_indices = np.where(train_labels_cifar.flatten() == i)[0]
    train_indices.extend(np.random.choice(class_indices, num_samples_per_class, replace=False))

train_images_simulated = train_images_cifar[train_indices]
train_labels_simulated = train_labels_cifar[train_indices]

# Usaremos o conjunto de teste original para avaliação, pois ele simula dados "não vistos"
test_images_simulated = test_images_cifar
test_labels_simulated = test_labels_cifar

print(f"Total de imagens de treino simuladas: {len(train_images_simulated)}")
print(f"Total de imagens de teste simuladas: {len(test_images_simulated)}")

# Mapear IDs de classe para nomes de classe (para melhor visualização)
id2label = {i: str(i) for i in range(10)}
label2id = {str(i): i for i in range(10)}

# --- 2. Pré-processamento e Aumento de Dados para ViT ---
# O ViT requer imagens de 224x224 (para o modelo vit-base-patch16-224-in21k).
# As transformações são CRUCIAIS para bases pequenas.

# Carrega o feature extractor do ViT pré-treinado
# Ele cuidará da normalização específica (mean/std) e do redimensionamento para 224x224
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# Definir as transformações de Data Augmentation para o treino
# Essas são transformações AVANÇADAS e importantes para bases pequenas!
train_transforms = transforms.Compose([
    transforms.ToPILImage(), # Converte numpy array para PIL Image para as transforms
    transforms.RandomResizedCrop(feature_extractor.size["height"]), # Alterado para acessar o valor do dicionário
    transforms.RandomHorizontalFlip(), # Espelhamento horizontal
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1), # Variação de cor
    transforms.ToTensor(), # Converte para tensor PyTorch
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std), # Normalização específica do ViT
])

# Definir as transformações para o conjunto de validação/teste (apenas redimensionamento e normalização)
test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(feature_extractor.size["height"]), # Alterado para acessar o valor do dicionário
    transforms.CenterCrop(feature_extractor.size["height"]), # Alterado para acessar o valor do dicionário
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Criar uma classe Dataset customizada para PyTorch
class ImageClassificationDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return {"pixel_values": image, "labels": torch.tensor(label, dtype=torch.long).flatten()}

# Criar os datasets PyTorch
train_dataset = ImageClassificationDataset(train_images_simulated, train_labels_simulated, train_transforms)
test_dataset = ImageClassificationDataset(test_images_simulated, test_labels_simulated, test_transforms)

# Converter para o formato Dataset do Hugging Face (opcional, mas facilita o Trainer)
train_dataset_hf = HfDataset.from_dict({"pixel_values": [np.array(img) for img in train_dataset.images], "labels": train_dataset.labels.flatten().tolist()})
test_dataset_hf = HfDataset.from_dict({"pixel_values": [np.array(img) for img in test_dataset.images], "labels": test_dataset.labels.flatten().tolist()})

# Aplicar o pré-processamento do feature_extractor
def preprocess_dataset(examples):
    # O feature_extractor aceita imagens no formato PIL Image ou numpy array
    images = [tf.keras.preprocessing.image.array_to_img(img) for img in examples["pixel_values"]]
    inputs = feature_extractor(images, return_tensors="pt")
    inputs["labels"] = examples["labels"]
    return inputs

# Não, é melhor usar o Dataset customizado com transforms do torchvision para ter mais controle sobre augmentation
# train_dataset_processed = train_dataset_hf.map(preprocess_dataset, batched=True)
# test_dataset_processed = test_dataset_hf.map(preprocess_dataset, batched=True)
# train_dataset_processed.set_format(type="torch", columns=["pixel_values", "labels"])
# test_dataset_processed.set_format(type="torch", columns=["pixel_values", "labels"])


# --- 3. Carregar o Modelo ViT para Fine-tuning ---
# Carregar o modelo ViT pré-treinado (com cabeça de classificação para 10 classes)
# O `num_labels` ajusta automaticamente a última camada de classificação.
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=10,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True # Permite ajustar a camada de saída
)

# Enviar o modelo para a GPU
model.to(device)

# --- 4. Configurar o Treinador (Trainer do Hugging Face) ---
# Métricas de avaliação
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./vit-fine-tuned-cifar10", # Diretório para salvar checkpoints
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10, # Aumente se necessário, mas monitore overfitting
    learning_rate=2e-5, # Taxa de aprendizado pequena para fine-tuning
    evaluation_strategy="epoch", # Avaliar a cada época
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch", # Salvar checkpoint a cada época
    load_best_model_at_end=True, # Carregar o melhor modelo no final
    metric_for_best_model="accuracy",
    report_to="none", # Não reportar para ferramentas externas (wandb, etc.)
)

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # Usar o dataset PyTorch customizado
    eval_dataset=test_dataset,   # Usar o dataset PyTorch customizado
    compute_metrics=compute_metrics,
    data_collator=lambda data: {
        'pixel_values': torch.stack([d['pixel_values'] for d in data]),
        'labels': torch.stack([d['labels'] for d in data])
    }
)

# --- 5. Realizar o Fine-tuning ---
print("\n--- Iniciando o Fine-tuning do ViT ---")
trainer.train()

# --- 6. Avaliar o Modelo Final ---
print("\n--- Avaliando o Modelo Final ---")
results = trainer.evaluate(test_dataset)
print(f"Resultados da Avaliação: {results}")

# Fazer previsões no conjunto de teste para a matriz de confusão
predictions_raw = trainer.predict(test_dataset)
y_pred = np.argmax(predictions_raw.predictions, axis=1)
y_true = np.array([example["labels"].item() for example in test_dataset]) # Extrair rótulos verdadeiros

conf_matrix = confusion_matrix(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

print(f"\nAcurácia do ViT Fine-Tuning no teste: {accuracy * 100:.2f}%")
print("\nMatriz de Confusão (ViT Fine-Tuning):")
print(conf_matrix)

# Visualização da Matriz de Confusão
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(id2label.values()), yticklabels=list(id2label.values()))
plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão (ViT Fine-Tuning)')
plt.show()

# --- Dicas Adicionais para Melhorar (e Lidar com Poucos Dados) ---
print("\n--- Dicas Adicionais para Melhorar (e Lidar com Poucos Dados) ---")
print("1. **Aumentar o número de épocas:** Com um dataset pequeno, pode ser que o modelo precise de mais épocas para convergir, mas cuidado com overfitting.")
print("2. **Ajustar a Learning Rate:** Experimente valores um pouco menores (e.g., 1e-6) ou use um scheduler de learning rate mais sofisticado.")
print("3. **Explorar mais Data Augmentation:** Se a acurácia não for satisfatória, adicione mais transformações complexas (ex: AutoAugment/RandAugment).")
print("4. **Tentar descongelar algumas camadas:** Se a acurácia estiver baixa, você pode tentar descongelar as últimas camadas do ViT (deixando `model.vit.encoder.layer[-N:].trainable = True`) e continuar o fine-tuning com uma learning rate ainda menor.")
print("5. **Usar um ViT menor:** Para bases muito pequenas, um ViT menor (ex: `google/vit-tiny-patch16-224-in21k`) pode ser menos propenso a overfitting, embora possa ter menor capacidade.")

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manyl

AttributeError: module 'torch.cuda' has no attribute 'enable_gpu'